<a href="https://colab.research.google.com/github/kundajelab/label_shift_experiments/blob/master/diabetic_retinopathy/KaggleOnlyDiabeticRetinopathyDomainAdaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://zenodo.org/record/3412990/files/kaggle_dr.txt.gz?download=1 -O kaggle_dr.txt.gz

--2019-09-24 01:47:18--  https://zenodo.org/record/3412990/files/kaggle_dr.txt.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106643 (104K) [application/octet-stream]
Saving to: ‘kaggle_dr.txt.gz’

kaggle_dr.txt.gz    100%[===================>] 104.14K  --.-KB/s    in 0.06s   

2019-09-24 01:47:19 (1.71 MB/s) - ‘kaggle_dr.txt.gz’ saved [106643/106643]



In [0]:
import gzip
import numpy as np

orig_kaggle_onehot_labels = [] #5 classes
orig_kaggle_predlogits = []
for row in gzip.open("kaggle_dr.txt.gz"):
    _, kaggle_label, logits = row.decode("utf-8").rstrip().split("\t")
    kaggle_label = int(kaggle_label)
    logits = [float(x) for x in logits.split(",")]
    labelsvec = np.zeros(5)
    labelsvec[kaggle_label] = 1.0
    orig_kaggle_onehot_labels.append(labelsvec)
    orig_kaggle_predlogits.append(logits)
orig_kaggle_onehot_labels = np.array(orig_kaggle_onehot_labels)
orig_kaggle_predlogits = np.array(orig_kaggle_predlogits)

In [3]:
from sklearn.metrics import roc_auc_score
from scipy.special import softmax

print(roc_auc_score(y_true=1-orig_kaggle_onehot_labels[:,0],
                    y_score=1-softmax(orig_kaggle_predlogits, axis=-1)[:,0]))

0.8945151902722942


In [4]:
![[ -e abstention ]] || git clone https://github.com/blindauth/abstention
%cd /content/abstention
!git pull
!pip uninstall abstention
!pip install .
%cd ..

/content/abstention
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/blindauth/abstention
   ba2f71c..d19d12a  master     -> origin/master
Updating ba2f71c..d19d12a
Fast-forward
 abstention/figure_making_utils.py | 10 ----------
 1 file changed, 10 deletions(-)
Uninstalling abstention-0.1.3.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/abstention-0.1.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/abstention/*
Proceed (y/n)? y
  Successfully uninstalled abstention-0.1.3.0
Processing /content/abstention
  Created wheel for abstention: filename=abstention-0.1.3.0-cp36-none-any.whl size=46751 sha256=377a03cad025d373cc6b213cb0d9a3531b8fd63eafbee1527160c8a2e04c7a06
  Stored in directory: /tmp/pip-ephem-wheel-cache-ogbn8ue6/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dc

In [5]:
![[ -e label_shift_experiments ]] || git clone https://github.com/blindauth/labelshiftexperiments
%cd /content/labelshiftexperiments
!git pull
!pip uninstall labelshiftexperiments
!pip install .
%cd ..

fatal: destination path 'labelshiftexperiments' already exists and is not an empty directory.
/content/labelshiftexperiments
Already up to date.
Uninstalling labelshiftexperiments-0.1.0.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/labelshiftexperiments-0.1.0.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/labelshiftexperiments/*
Proceed (y/n)? y
  Successfully uninstalled labelshiftexperiments-0.1.0.0
Processing /content/labelshiftexperiments
  Created wheel for labelshiftexperiments: filename=labelshiftexperiments-0.1.0.0-cp36-none-any.whl size=6278 sha256=60bf39581f95722b5f9c24cbbaadc6c65df17d2d3946a1317c7bbd26dc05750f
  Stored in directory: /tmp/pip-ephem-wheel-cache-lt9aldwx/wheels/6d/56/a7/fc736645f87832f3f9af94ef6661744069f41cd66bab35ec1b
Successfully built labelshiftexperiments
/content


In [0]:
#do calibration and domain adaptation
from collections import OrderedDict
from importlib import reload
import abstention
reload(abstention)
reload(abstention.calibration)
reload(abstention.label_shift)
from abstention.calibration import (
    TempScaling, NoBiasVectorScaling, VectorScaling, CrossValidatedBCTS, softmax)
from abstention.label_shift import (EMImbalanceAdapter, BBSEImbalanceAdapter)

calibname_to_calibfactory = OrderedDict([
    ('None', abstention.calibration.Softmax()),
    ('TS', TempScaling(verbose=False)),
    ('VS', VectorScaling(verbose=False)),
    ('NBVS', NoBiasVectorScaling(verbose=False)),
    ('BCTS', TempScaling(verbose=False,
                         bias_positions=[0,1,2,3,4])),
    #('CVBCTS', CrossValidatedBCTS(verbose=True,
    #                              num_crossvalidation_splits=5,
    #                              betas_to_try=[0.0, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10]
    #                             ))
])

adaptname_to_adapter = OrderedDict([
    ('em', EMImbalanceAdapter()),
    ('bbse-hard', BBSEImbalanceAdapter(soft=False)),
    ('bbse-soft', BBSEImbalanceAdapter(soft=True))
])
    
adaptncalib_pairs = [
    ('bbse-hard', 'None'),
    ('bbse-soft', 'None'),
    ('bbse-soft', 'TS'),
    ('bbse-soft', 'NBVS'),
    ('bbse-soft', 'BCTS'),
    ('bbse-soft', 'VS'),
    ('bbse-soft', 'best-ece'),
    ('bbse-soft', 'best-jsdiv'),
    ('bbse-soft', 'best-nll'),
    #('bbse-soft', 'CVBCTS'),
    ('em', 'None'),
    ('em', 'TS'),
    ('em', 'NBVS'),
    ('em', 'BCTS'),
    ('em', 'VS'),
    ('em', 'best-ece'),
    ('em', 'best-jsdiv'),
    ('em', 'best-nll')
    #('em', 'CVBCTS'),
]
    

In [7]:
len(orig_kaggle_onehot_labels)

3514

In [8]:
np.mean(orig_kaggle_onehot_labels, axis=0)

array([0.7310757 , 0.07057484, 0.15253273, 0.02447353, 0.0213432 ])

In [10]:
from collections import defaultdict
import numpy as np
import sys
from importlib import reload
import abstention
reload(abstention)
reload(abstention.calibration)
reload(abstention.label_shift)
from abstention.calibration import (
    TempScaling, #CrossValidatedBCTS,
    ConfusionMatrix, softmax)
from abstention.label_shift import (EMImbalanceAdapter,
     BBSEImbalanceAdapter, ShiftWeightFromImbalanceAdapter)
import glob
import numpy as np
from collections import defaultdict, OrderedDict
import scipy

import labelshiftexperiments
reload(labelshiftexperiments)
reload(labelshiftexperiments.cifarandmnist)
from labelshiftexperiments import cifarandmnist

metric_to_adaptname_to_perfs = defaultdict(lambda: defaultdict(list))

num_trials = 100
tweakone_alphas_and_samplesize = [(0.5,500), (0.9,500),
                                  (0.5, 1000), (0.9, 1000),
                                  (0.5,1500), (0.9,1500)]

tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals = (
        defaultdict(
            lambda: defaultdict(
                     lambda: defaultdict(
                              lambda: defaultdict(list)))))
tweakone_alpha_to_samplesize_to_baselineperf = defaultdict(lambda: defaultdict(list))
metric_to_samplesize_to_calibname_to_unshiftedvals = defaultdict(
    lambda: defaultdict(lambda: defaultdict(list)))
samplesizesseen = set()

orig_class_priors = np.mean(orig_kaggle_onehot_labels, axis=0)

for (alpha, samplesize) in tweakone_alphas_and_samplesize:
    for trail_num in range(num_trials):
        rng = np.random.RandomState(trail_num*10)
        #sample a split of pseudovalid/pseudotest indices
        indices = rng.choice(np.arange(len(orig_kaggle_onehot_labels)),
                             size=len(orig_kaggle_onehot_labels),
                             replace=False)
        pseudovalid_indices = indices[:samplesize]
        #calibrateonme_indices = indices[samplesize:2*samplesize]
        pseudotest_indices = indices[samplesize:]
        assert ((len(pseudovalid_indices)+len(pseudotest_indices))
                ==len(orig_kaggle_onehot_labels))
        
        pseudovalid_preacts = orig_kaggle_predlogits[pseudovalid_indices]
        pseudotest_preacts = orig_kaggle_predlogits[pseudotest_indices]
        pseudovalid_labels = orig_kaggle_onehot_labels[pseudovalid_indices]
        pseudotest_labels = orig_kaggle_onehot_labels[pseudotest_indices]
        
        draw_test_indices = cifarandmnist.get_func_to_draw_label_proportions(
                              test_labels=pseudotest_labels)
        
        #perform calibration
        (calibname_to_calibfunc,
         calibname_to_calibpseudovalidpreds) = (
                    cifarandmnist.run_calibmethods(
                       valid_preacts=pseudovalid_preacts,
                       valid_labels=pseudovalid_labels,
                       test_preacts=pseudotest_preacts,
                       test_labels=pseudotest_labels,
                       calibname_to_calibfactory=
                        calibname_to_calibfactory,
                       samplesize=samplesize,
                       samplesizesseen=samplesizesseen,
                       metric_to_samplesize_to_calibname_to_unshiftedvals=
                        metric_to_samplesize_to_calibname_to_unshiftedvals))
        
        #note the calibration method that did the best according to each metric,
        # and save it
        for metricname in metric_to_samplesize_to_calibname_to_unshiftedvals:
            calibname_to_unshiftedvals = metric_to_samplesize_to_calibname_to_unshiftedvals[metricname][samplesize]
            best_calibname = min(list(calibname_to_unshiftedvals.keys()),
                                 key=lambda x: calibname_to_unshiftedvals[x][-1])
            calibname_to_calibfunc['best-'+metricname] = calibname_to_calibfunc[best_calibname]
            calibname_to_calibpseudovalidpreds['best-'+metricname] = calibname_to_calibpseudovalidpreds[best_calibname]
        
        #perform the knockout shift
        altered_class_priors = np.zeros(5)
        altered_class_priors[0] = alpha
        altered_class_priors[1:] = (1-alpha)*orig_class_priors[1:]
        
        pseudotest_indices = draw_test_indices(
                          total_to_return=samplesize,                    
                          label_proportions=altered_class_priors,        
                          rng=rng)
        shifted_pseudotest_labels = pseudotest_labels[pseudotest_indices]
        shifted_pseudotest_preacts = pseudotest_preacts[pseudotest_indices]
        
        calibname_to_calibshiftedtestpreds = {}
        for (calibname, calibfunc) in calibname_to_calibfunc.items():
            calibname_to_calibshiftedtestpreds[calibname] =(
                calibfunc(shifted_pseudotest_preacts))
        
        shifted_pseudotest_baseline_perf = np.mean(
            np.argmax(shifted_pseudotest_preacts, axis=-1)
            ==np.argmax(shifted_pseudotest_labels, axis=-1))
        tweakone_alpha_to_samplesize_to_baselineperf[alpha][samplesize].append(
            shifted_pseudotest_baseline_perf)
        true_shifted_priors = np.mean(shifted_pseudotest_labels, axis=0)
        
        for adapter_name,calib_name in adaptncalib_pairs:
            calib_shifted_pseudotest_preds =\
              calibname_to_calibshiftedtestpreds[calib_name]
            calib_pseudovalid_preds =\
              calibname_to_calibpseudovalidpreds[calib_name]
            imbalance_adapter = adaptname_to_adapter[adapter_name]
            imbalance_adapter_func = imbalance_adapter(
                valid_labels=pseudovalid_labels,                       
                tofit_initial_posterior_probs=calib_shifted_pseudotest_preds, 
                valid_posterior_probs=calib_pseudovalid_preds)
            adapted_shifted_pseudotest_preds = imbalance_adapter_func(        
                        calib_shifted_pseudotest_preds)
            estim_shifted_priors = np.mean(adapted_shifted_pseudotest_preds,  
                                           axis=0)
            #compute metrics for quality of shift
            adapted_shifted_pseudotest_perf = np.mean(
              np.argmax(adapted_shifted_pseudotest_preds, axis=-1)
              ==np.argmax(shifted_pseudotest_labels, axis=-1))
            delta_from_baseline = (adapted_shifted_pseudotest_perf-
                                   shifted_pseudotest_baseline_perf)
            
            tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[
                alpha][samplesize][adapter_name+":"+calib_name][
                'jsdiv'].append(
                  scipy.spatial.distance.jensenshannon(
                      p=true_shifted_priors, q=estim_shifted_priors))
            tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[
                alpha][samplesize][adapter_name+":"+calib_name][
                'delta_perf'].append(delta_from_baseline)
        
    if (samplesize not in samplesizesseen):
        print("Calibration stats")
        for metric in ['ece', 'nll', 'jsdiv']:
            print("Metric",metric)
            for calibname in calibname_to_calibfactory:
                print(calibname, np.mean(                     
                       metric_to_samplesize_to_calibname_to_unshiftedvals[
                        metric][samplesize][calibname]))  
        samplesizesseen.add(samplesize)

    print("On alpha",alpha,"sample size", samplesize)
    for metric_name in ['delta_perf', 'jsdiv']: 
        print("Metric",metric_name)
        for adapter_name,calib_name in adaptncalib_pairs:
            adaptncalib_name = adapter_name+":"+calib_name
            n = len(tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[
                  alpha][samplesize][adaptncalib_name][metric_name])

            print(adaptncalib_name, np.mean(
             tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[
               alpha][samplesize][adaptncalib_name][metric_name]), "+/-",
             (1.0/np.sqrt(n))*np.std(
             tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[
               alpha][samplesize][adaptncalib_name][metric_name],
               ddof=1))           
            sys.stdout.flush()

Calibration stats
Metric ece
None 8.733662753798031
TS 3.649580871054098
VS 2.3231526172283683
NBVS 2.1297461941216516
BCTS 2.2547945348797995
Metric nll
None 0.6402319025011183
TS 0.5711691961104738
VS 0.5180747116069775
NBVS 0.5433706417895835
BCTS 0.5144445766964391
Metric jsdiv
None 0.07661477273081052
TS 0.09092214616308848
VS 0.025921436855378746
NBVS 0.045375127921321035
BCTS 0.02565906567873681
On alpha 0.5 sample size 500
Metric delta_perf
bbse-hard:None -0.026600000000000026 +/- 0.01381732187648474
bbse-soft:None 0.003419999999999965 +/- 0.007343645366759433
bbse-soft:TS 0.003359999999999972 +/- 0.004355595959408558
bbse-soft:NBVS 0.009099999999999962 +/- 0.0076981632336367845
bbse-soft:BCTS 0.015599999999999961 +/- 0.009696859847862652
bbse-soft:VS 0.02517999999999996 +/- 0.002664710646258393
bbse-soft:best-ece 0.020479999999999957 +/- 0.0029341761874485126
bbse-soft:best-jsdiv 0.02527999999999996 +/- 0.002753072847022049
bbse-soft:best-nll 0.015459999999999963 +/- 0.0096928

In [11]:
import json
import os
file_out = "label_shift_adaptation_results.json"
dict_to_write = {
    "tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals":
     tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    "tweakone_alpha_to_samplesize_to_baselineperf":
     tweakone_alpha_to_samplesize_to_baselineperf,
    "metric_to_samplesize_to_calibname_to_unshiftedvals":
     metric_to_samplesize_to_calibname_to_unshiftedvals
}
open(file_out, 'w').write(
    json.dumps(dict_to_write,
               sort_keys=True, indent=4, separators=(',', ': ')))
os.system("gzip -f "+file_out)

0

In [0]:
import gzip
import json
loaded_dicts = json.loads(gzip.open("label_shift_adaptation_results.json.gz").read())
tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals =\
    loaded_dicts['tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals']
tweakone_alpha_to_samplesize_to_baselineperf =\
    loaded_dicts['tweakone_alpha_to_samplesize_to_baselineperf']
metric_to_samplesize_to_calibname_to_unshiftedvals =\
    loaded_dicts['metric_to_samplesize_to_calibname_to_unshiftedvals']

In [17]:
from importlib import reload
import labelshiftexperiments
reload(labelshiftexperiments)
import abstention.figure_making_utils
reload(abstention.figure_making_utils)
import labelshiftexperiments.maketable
reload (labelshiftexperiments.maketable)
from labelshiftexperiments.maketable import render_calibration_table

metricname_to_nicename = {'nll': 'nll', 'jsdiv': 'jsdiv', 'ece': 'ECE'}
calibname_to_nicename = {'None': "None", "TS": "TS",
                         "VS":"VS", "NBVS": "NBVS", "BCTS": "BCTS"}
  
from scipy.stats import norm
N = num_trials
#Using the normal approximation at N=100;
# variance from https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test
#Note that T = ((N+1)*N/2 - W)/2            
ustat_threshold = ((N*(N+1))/2 - norm.ppf(0.99)*np.sqrt(N*(N+1)*(2*N+1)/6.0))/2.0
  
print(render_calibration_table(
        metric_to_samplesize_to_calibname_to_unshiftedvals=
          metric_to_samplesize_to_calibname_to_unshiftedvals,
        #threshold of 8 comes from table https://www.oreilly.com/library/view/nonparametric-statistics-a/9781118840429/bapp02.xhtml
        #for one-tailed alpha=0.025 and n=10            
        ustat_threshold=ustat_threshold,
        metrics_in_table=['nll', 'jsdiv', 'ece'],
        samplesizes_in_table=['500', '1000', '1500'],
        calibnames_in_table=['None', 'TS', 'NBVS', 'BCTS', 'VS'],
        metricname_to_nicename=metricname_to_nicename,
        calibname_to_nicename=calibname_to_nicename,
        caption="KaggleDR Calibration metric differences", label="kaggledrcalibrationcomparison",
        applyunderline=False))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c c c | c c c | c c c }
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{nll} & \multicolumn{3}{| c}{jsdiv} & \multicolumn{3}{| c}{ECE}\\
    \cline{2-10}
    & $n$=500 & $n$=1000 & $n$=1500 & $n$=500 & $n$=1000 & $n$=1500 & $n$=500 & $n$=1000 & $n$=1500\\
    \hline
    None & 0.64; 4.0 & 0.639; 4.0 & 0.639; 4.0 & 0.077; 2.99 & 0.077; 3.0 & 0.077; 3.0 & 8.734; 4.0 & 8.737; 4.0 & 8.767; 4.0\\
    TS & 0.571; 3.0 & 0.57; 3.0 & 0.569; 3.0 & 0.091; 4.0 & 0.091; 4.0 & 0.091; 4.0 & 3.65; 2.77 & 3.729; 2.92 & 3.853; 2.76\\
    NBVS & 0.543; 2.0 & 0.54; 2.0 & 0.539; 2.0 & 0.045; 1.85 & 0.044; 1.95 & 0.044; 1.98 & \textbf{2.13; 0.67} & \textbf{2.028; 0.97} & \textbf{2.129; 1.01}\\
    BCTS & \textbf{0.514; 0.21} & \textbf{0.511; 0.57} & 0.511; 0.63 & \textbf{0.026; 0.62} & \textbf{0.02; 0.48} & \textbf{0.018; 0.45} & 2.255; 1.21 & \textbf{2.09

In [20]:
from labelshiftexperiments.maketable import render_adaptation_table
    
methodgroups = OrderedDict([
  ('em', ['em:None', 'em:TS', 'em:NBVS', 'em:BCTS', 'em:VS']),
  ('bbse', ['bbse-hard:None', 'bbse-soft:None',
           'bbse-soft:TS', 'bbse-soft:NBVS', 'bbse-soft:BCTS', 'bbse-soft:VS'])])
metric_to_largerisbetter = {'perf':True}
samplesizes_in_table = ['500', '1000', '1500']
adaptname_to_nicename = {'em': 'EM',
                           'bbse-soft': 'BBSE-soft',
                           'bbse-hard': 'BBSE-hard'}
calibname_to_nicename = {'None': 'None',
                           'TS': 'TS',
                           'NBVS': 'NBVS',
                           'BCTS': 'BCTS',
                           'VS': 'VS',
                           'best-nll':'Best NLL',
                           'best-jsdiv':'Best JS Div',
                           'best-ece':'Best ECE',}

tweakone_alphas_in_table = ['0.5', '0.9']

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=1.0,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='jsdiv',
    largerisbetter=False,
    alphas_in_table=tweakone_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="KaggleDR Metric: JS Divergence, tweakone shift",
    label="kaggledrjsdivtweakoneshift",
    applyunderline=False))

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_perf',
    largerisbetter=True,
    alphas_in_table=tweakone_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="KaggleDR Metric: $\\Delta$\\%Accuracy, tweakone shift",
    label="kaggledrdeltaacctweakoneshift",
    applyunderline=False))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c | c c c | c c c}
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Shift} \\ \textbf{Estimator} \end{tabular}} & \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{$\alpha=0.5$} & \multicolumn{3}{| c}{$\alpha=0.9$}\\ 
    \cline{3-8}
    & & $n$=500 & $n$=1000 & $n$=1500 & $n$=500 & $n$=1000 & $n$=1500\\
    \hline
    \hline
    EM & None & \textbf{0.107; 1.62} & 0.102; 2.19 & 0.095; 2.25 & 0.098; 1.89 & 0.087; 1.9 & 0.087; 1.8\\
    EM & TS & 0.126; 2.3 & 0.108; 2.21 & 0.101; 2.29 & 0.133; 3.64 & 0.122; 3.79 & 0.123; 3.82\\
    EM & NBVS & 0.117; 2.32 & \textbf{0.095; 2.06} & 0.087; 2.15 & 0.118; 2.83 & 0.108; 2.67 & 0.109; 2.85\\
    EM & BCTS & \textbf{0.111; 1.94} & 0.095; 2.0 & 0.082; 1.85 & \textbf{0.078; 0.95} & \textbf{0.065; 1.0} & 0.063; 0.87\\
    EM & VS & \textbf{0.108; 1.82} & \textbf{0.088; 1.54} & \textbf{0.077; 1.46} & \textb

In [21]:
methodgroups = OrderedDict([
  ('em-calib', ['em:best-nll', 'em:best-jsdiv', 'em:best-ece']),
  ('bbse-calib', ['bbse-soft:best-nll', 'bbse-soft:best-jsdiv', 'bbse-soft:best-ece'])])

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_perf',
    largerisbetter=True,
    alphas_in_table=tweakone_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="KaggleDR Metric: $\\Delta$\\%Accuracy, tweakone shift",
    label="kaggledrdeltaacctweakoneshift",
    applyunderline=False))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c | c c c | c c c}
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Shift} \\ \textbf{Estimator} \end{tabular}} & \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{$\alpha=0.5$} & \multicolumn{3}{| c}{$\alpha=0.9$}\\ 
    \cline{3-8}
    & & $n$=500 & $n$=1000 & $n$=1500 & $n$=500 & $n$=1000 & $n$=1500\\
    \hline
    \hline
    EM & Best NLL & \textbf{3.79; 0.36} & 4.315; 0.51 & 4.543; 0.4 & \textbf{3.548; 0.02} & \textbf{3.57; 0.0} & \textbf{3.746; 0.38}\\
    EM & Best JS Div & 3.788; 1.13 & \textbf{4.321; 0.97} & 4.526; 0.96 & \textbf{3.548; 1.02} & \textbf{3.57; 1.0} & \textbf{3.722; 0.65}\\
    EM & Best ECE & 3.49; 1.51 & 4.099; 1.52 & 4.179; 1.64 & 2.074; 1.96 & \textbf{3.57; 2.0} & 2.405; 1.97\\
    \hline
    \hline
    BBSE-soft & Best NLL & 1.546; 0.38 & 3.584; 0.52 & \textbf{4.164; 0.38} & \textbf{3.282; 0.07} & \textbf{3.442; 0.